### Importing Libraries

In [22]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import encode_columns, svm_model, accuracy_calculator, random_forest_model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import ast

### Loading the dataset using Pandas
The data is found in development.csv (development set): a comma-separated values file containing the records from the development set. This portion does have the action and object columns, which you should use to obtain the labels to train and validate your models.
The dataset consists in a collection of audio file in a WAV format. 

Each record is characterized by several attributes. The following is a short description for each of them.
- path: the path of the audio file.
- speakerId: the id of the speaker.
- action: the type of action required through the intent.
- object: the device involved by intent.
- Self-reported fluency level: the speaking fluency of the speaker.
- First Language spoken: the first language spoken by the speaker.
- Current language used for work/school: the main language spoken by the speaker during daily activities.
- gender: the gender of the speaker.
- ageRange: the age range of the speaker.

In [23]:
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")

In [24]:
y = df.values[:,3:5].sum(axis=1)
y

array(['change languagenone', 'activatemusic', 'deactivatelights', ...,
       'deactivatelights', 'deactivatelights', 'increasevolume'],
      dtype=object)

In [25]:
column_names = ["Self-reported fluency level ","First Language spoken", "Current language used for work/school", "speakerId", "gender","ageRange"]


In [26]:
encode_columns(df_eval, column_names)

In [27]:

encode_columns(df,column_names)

In [28]:
df_eval["mfccs"] = df_eval.apply(lambda row: librosa.feature.mfcc(y=librosa.load(row["path"])[0], sr=librosa.load(row["path"])[1]), axis=1)

In [29]:
df["mfccs"] = df.apply(lambda row: librosa.feature.mfcc(y=librosa.load(row["path"])[0], sr=librosa.load(row["path"])[1]), axis=1)

In [30]:
for i, row in df_eval.iterrows():
    for j, array in enumerate(row['mfccs']):
        mean_array, std_array = np.mean(array), np.std(array)
        df_eval.at[i, f'mfcc_mean{j+1}'] = mean_array

In [31]:
for i, row in df.iterrows():
    for j, array in enumerate(row['mfccs']):
        mean_array, std_array = np.mean(array), np.std(array)
        df.at[i, f'mfcc_mean{j+1}'] = mean_array

In [32]:
features = ['Self-reported fluency level ',
       'First Language spoken', 'Current language used for work/school',
       'speakerId', 'gender', 'ageRange', 'mfcc_mean1', 'mfcc_mean2',
       'mfcc_mean3', 'mfcc_mean4', 'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7',
       'mfcc_mean8', 'mfcc_mean9', 'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12',
       'mfcc_mean13', 'mfcc_mean14', 'mfcc_mean15', 'mfcc_mean16',
       'mfcc_mean17', 'mfcc_mean18', 'mfcc_mean19', 'mfcc_mean20']

In [33]:
X_eval = df_eval[features].copy()
X_eval

,Self-reported fluency level,First Language spoken,Current language used for work/school,speakerId,gender,ageRange,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,...,mfcc_mean11,mfcc_mean12,mfcc_mean13,mfcc_mean14,mfcc_mean15,mfcc_mean16,mfcc_mean17,mfcc_mean18,mfcc_mean19,mfcc_mean20
0,0,0,0,2,1,0,-527.157288,85.485222,24.571247,8.796556,...,4.440116,2.022582,-3.226901,-0.751710,1.739519,-2.511429,-5.771698,-3.176437,-2.367861,-5.021607
1,0,0,0,7,1,0,-370.422455,70.736412,0.093442,50.700577,...,-12.636522,-20.866716,-6.486042,-7.064291,-14.025457,-4.765747,-7.965150,-5.508674,0.478870,-7.206638
2,0,0,0,1,1,1,-521.495544,93.595200,-45.410805,9.142069,...,-4.268334,-5.545475,-4.384758,-5.988326,-4.071120,-3.235104,-1.026856,-7.564101,-6.578736,-3.657175
3,0,0,0,7,1,0,-431.347015,69.204933,2.856670,25.795380,...,-0.446622,-16.987175,-2.079778,-0.257614,-9.580922,-9.229519,-5.560866,-4.248737,-4.731292,-6.816913
4,0,0,0,4,0,1,-384.823517,75.228111,-31.649582,37.719921,...,-1.103042,-4.541669,-1.965617,-6.189669,4.167395,-6.394841,-0.512685,-3.434928,-3.147377,-1.278373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450,0,0,0,8,0,0,-407.407532,89.597038,-5.482538,33.206337,...,0.733915,-9.134067,1.504485,-5.733008,3.598521,-11.799407,-3.572668,-1.405530,-5.543880,-4.082631
1451,0,0,0,8,0,0,-401.939880,95.584824,-8.089671,20.611967,...,5.813816,-5.453441,-6.012916,-7.026556,-2.340339,-6.943107,-4.205869,-4.611456,-3.682290,-5.553263
1452,0,0,0,9,0,1,-413.348206,35.094906,-38.453888,32.987171,...,4.643959,-9.835579,2.598943,0.757164,6.849001,-7.827218,3.865885,-2.916348,-4.098361,3.503630
1453,0,0,0,6,0,0,-401.751587,86.633377,-8.537997,37.541016,...,-8.759080,-16.995852,-8.195457,-7.113196,-16.877422,-14.724919,-4.436277,-7.502899,-9.664334,-9.862633


In [34]:

X = df[features].copy()
X

,Self-reported fluency level,First Language spoken,Current language used for work/school,speakerId,gender,ageRange,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,...,mfcc_mean11,mfcc_mean12,mfcc_mean13,mfcc_mean14,mfcc_mean15,mfcc_mean16,mfcc_mean17,mfcc_mean18,mfcc_mean19,mfcc_mean20
0,0,1,2,0,0,0,-508.360962,55.001785,-28.398556,58.829868,...,-1.449769,-6.053585,-2.133756,-8.021720,0.036106,-14.021687,5.120137,-12.067747,-6.652406,-0.046004
1,0,1,2,0,0,0,-595.742981,80.647659,6.302678,34.304688,...,-4.657745,-9.864026,-1.916556,-8.835615,-4.063780,-9.214926,-2.201957,-9.320792,-7.384116,-6.376321
2,0,1,2,0,0,0,-632.323181,117.935394,-6.713594,11.537451,...,-5.031530,-5.736244,5.455654,-8.862136,-2.851791,-7.492860,-1.596385,-6.176934,-6.506490,-1.477143
3,0,1,2,0,0,0,-527.010254,93.092232,-21.026457,16.220224,...,-1.631703,-4.324308,-4.468716,-7.331700,-5.664731,-4.863215,-4.570913,-8.062627,-4.719793,-8.513862
4,0,1,2,0,0,0,-667.823425,114.183678,10.975331,-2.496258,...,-2.299047,-5.092690,-2.289111,-5.000788,-6.487605,-5.174390,-2.906508,-2.341805,-2.721355,-2.526303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9849,3,1,2,76,1,0,-463.235168,86.606964,-7.117648,26.333221,...,0.128316,1.672917,3.883306,-4.701414,5.126201,-2.752902,-0.638182,3.083964,-0.046266,2.173160
9850,3,1,2,76,1,0,-467.034210,65.140160,-14.032764,31.461567,...,2.539551,1.458298,5.786525,-2.076686,7.531277,-1.950297,2.556846,4.036376,0.453042,1.127768
9851,3,1,2,76,1,0,-434.696594,82.030121,-20.899393,25.840460,...,3.016115,3.169316,4.748728,-4.774180,8.147912,-2.660758,-0.034383,2.312869,2.208149,1.832580
9852,3,1,2,76,1,0,-490.155182,63.094467,-9.643861,27.813547,...,2.231100,3.845147,4.476443,-2.501357,5.506608,-2.987334,1.936801,2.290256,1.500462,2.688512


In [35]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# y_pred_eval = svm_model(X, y, X_eval)

# predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
# predictions.to_csv('my_data.csv', index=True)

In [37]:
# y_pred = svm_model(X_train, y_train, X_test)
# svm_accuracy = accuracy_calculator(y_test, y_pred)

In [38]:
# svm_accuracy

In [39]:
y_pred_eval = random_forest_model(X , X_eval, y)

predictions = pd.DataFrame(y_pred_eval,columns=["Predicted"])
predictions.to_csv('predictions.csv', index=True)


In [40]:
# y_pred = random_forest_model(X_train, X_test, y_train)
# random_forest_accuracy = accuracy_calculator(y_test, y_pred)
# random_forest_accuracy

In [41]:
## Implementation using k-fold
from sklearn.model_selection import KFold, cross_val_score
# define the number of folds and whether to shuffle the data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# create an instance of the SVM model
clf = SVC()

# use cross_val_score function to perform k-fold cross-validation
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

# print the mean accuracy and standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

Accuracy: 0.31 (+/- 0.01)


In [42]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X, y = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)


Accuracy: 0.44631584369293853
